In [1]:
import pandas as pd

# 2016 Election Data
These csv files were downloaded on October 8th, 2020 from http://www.electproject.org/. This MySQL migration was done by Sarah Schulte 

## Primary 2016

In [2]:
Primary2016 = pd.read_csv("2016PrimaryElection2.csv")
# Primary2016

# State Lat/Long for future map use? 

In [3]:
state_json = "../state_pop.json"
state_df = pd.read_json(state_json)
# state_df

### Formatting DF 

In [4]:
NewHeader = Primary2016.iloc[0]
# NewHeader
Primary2016 = Primary2016[1:]
# Primary2016

In [5]:
Primary2016.columns = NewHeader
# Primary2016


### Cleaning
Dropping unnecessary columns and formating data types

In [6]:
Columns = Primary2016[['State','Voting-Eligible Population (VEP)','Voting-Age Population (VAP)']]
Primary2016Drop = Columns.copy()
Primary2016Drop.head()

,State,Voting-Eligible Population (VEP),Voting-Age Population (VAP)
1,Alabama,3602334,3764601
2,Alaska,520731,553484
3,Arizona,4659373,5268830
4,Arkansas,2137115,2281940
5,California,25126973,30367243


In [7]:
Primary2016Drop['State'].nunique()

51

In [8]:
# Primary2016Drop.dtypes

In [9]:
Primary2016Drop['Voting-Eligible Population (VEP)'] = Primary2016Drop['Voting-Eligible Population (VEP)'].astype(int)
# Primary2016Drop.dtypes

In [10]:
Primary2016Drop['Voting-Age Population (VAP)'] = Primary2016Drop['Voting-Age Population (VAP)'].astype(int)
# Primary2016Drop.dtypes

### Preview
For this table, DC is indexed as the 51st state

In [22]:
# Primary2016Drop

## General Election 2016


In [12]:
General2016 = pd.read_csv("2016GeneralElection.csv")
# General2016.head()

### Formatting DF

In [13]:
NewHeader = General2016.iloc[0]
# NewHeader

In [14]:
General2016 = General2016[1:]
# General2016

In [15]:
General2016.columns = NewHeader
General2016.head()

,State,Website,Status,VEP Total Ballots Counted,VEP Highest Office,VAP Highest Office,Total Ballots Counted (Estimate),Highest Office,Voting-Eligible Population (VEP),Voting-Age Population (VAP),% Non-citizen,Prison,Probation,Parole,Total Ineligible Felon,Overseas Eligible,State Abv
1,Alabama,http://www.alabamavotes.gov/downloads/election...,Official,59.10%,58.80%,56.30%,2134061,2123372,3609447,3770142,2.50%,28680,52177,8562,66983,NaN,AL
2,Alaska,http://www.elections.alaska.gov/results/16GENR/,Official,61.50%,61.00%,57.40%,321271,318608,522679,555367,4.10%,4317,6621,1812,9903,NaN,AK
3,Arizona,http://apps.azsos.gov/election/2016/General/Of...,Official,56.10%,54.90%,48.90%,2661497,2604657,4740310,5331034,9.40%,37582,77373,7500,89185,NaN,AZ
4,Arkansas,http://results.enr.clarityelections.com/AR/639...,Official,53.20%,52.80%,49.40%,1137772,1130635,2140097,2286625,3.80%,17242,30881,23792,58636,NaN,AR
5,California,http://www.sos.ca.gov/elections/prior-election...,Official,58.20%,56.50%,47.00%,14610509,14181595,25104844,30201571,16.40%,130390,0,0,130390,NaN,CA


### Cleaning

In [16]:
Columns = General2016[['State','Voting-Eligible Population (VEP)','Voting-Age Population (VAP)','State Abv']]
General2016Drop = Columns.copy()
General2016Drop.head()

,State,Voting-Eligible Population (VEP),Voting-Age Population (VAP),State Abv
1,Alabama,3609447,3770142,AL
2,Alaska,522679,555367,AK
3,Arizona,4740310,5331034,AZ
4,Arkansas,2140097,2286625,AR
5,California,25104844,30201571,CA


In [17]:
# General2016Drop.dtypes

0
State                               object
Voting-Eligible Population (VEP)    object
Voting-Age Population (VAP)         object
State Abv                           object
dtype: object

In [23]:
General2016Drop['Voting-Eligible Population (VEP)'] = General2016Drop['Voting-Eligible Population (VEP)'].astype(int)
General2016Drop['Voting-Age Population (VAP)'] = General2016Drop['Voting-Age Population (VAP)'].astype(int)
# General2016Drop.dtypes

### Preview Table

In [20]:
# General2016Drop

# Migrating to MySQL Database

In [25]:
from sqlalchemy import create_engine
import pymysql
import json

from config import remote_db_endpoint, remote_db_port, remote_db_user, remote_db_pwd, remote_db_name

In [26]:
pymysql.install_as_MySQLdb()
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}/{remote_db_name}")
cxn = engine.connect()

In [27]:
# Primary2016Drop.to_sql('2016_primary_election', cxn, if_exists = 'replace', index = False)

In [28]:
# General2016Drop.to_sql('2016_general_election', cxn, if_exists = 'replace', index = False)

## Checking the migration

In [33]:
aws_check = pd.read_sql("SELECT * FROM 2016_primary_election", cxn)
# aws_check

In [34]:
aws_check = pd.read_sql("SELECT * FROM 2016_general_election", cxn)
# aws_check